<a href="https://colab.research.google.com/github/AVICII-24/Exo-planet_Detect_SEDSCelestia/blob/main/Exoplanet_code_iter2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import numpy as np
import tensorflow as tf
import pandas as pd

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

from sklearn.preprocessing import normalize

# import the dataset
data_train = pd.read_csv('/content/exoTrain.csv')
data_test = pd.read_csv('/content/exoTest.csv')

#Checking to see label distribution
if 'LABEL' in data_train.columns:
    # Count the occurrences of 1 and 2 in the 'label' column
    count_1 = (data_train['LABEL'] == 1).sum()
    count_2 = (data_train['LABEL'] == 2).sum()

    print(f"Number of times 1 appears in the 'label' column: {count_1}")
    print(f"Number of times 2 appears in the 'label' column: {count_2}")
'''For Our Reference:'''
#Number of times 1 appears in the 'label' column: 5050
#Number of times 2 appears in the 'label' column: 37
'''
also remember:
1: not an exoplanet
2: is an exoplanet
'''



Number of times 1 appears in the 'label' column: 5050
Number of times 2 appears in the 'label' column: 37


'\nalso remember:\n1: not an exoplanet\n2: is an exoplanet\n'

In [8]:
#Run this cell only if you want to generate a 500 record csv file. [seperate from the rest of the code]
#Currently im underscaling the usable dataset, only going to be using about 500 values in total, 37 (LABEL=2)*3 and rest(389) as random LABEL=1's
import csv

# Open the new CSV file for writing
with open("/content/500size_dataset_exoplanet.csv", 'w', newline='') as f1:
    writer = csv.writer(f1)
    writer.writerow(data_train.columns)
    num_1s, num_2s = 389, 37
    for _, row in data_train.iterrows():
        if row['LABEL'] == 1 and num_1s > 0:
            writer.writerow(row)
            num_1s -= 1
        elif row['LABEL'] == 2 and num_2s > 0:
            for _ in range(3):  # Oversampling positive indicators by 3 times
                writer.writerow(row)
            num_2s -= 1

        if num_1s == 0 and num_2s == 0:
            break

print("500-size dataset CSV file created successfully!")


500-size dataset CSV file created successfully!


In [13]:
'''New data_train_500 specified on the basis of the new file'''
data_train_500 = pd.read_csv('/content/exoTrain.csv')

#permute the dataset
data_train_500 = np.random.permutation(np.asarray(data_train_500))
data_test = np.random.permutation(np.asarray(data_test))
#normalized data
data_train_500_norm = normalize(data_train_500)
data_test_norm = normalize(data_test)

#Now we have normalised 500 datapoints for training, and the entire testing dataset(we can even shorten the testing data set to reduce runtime...)


In [17]:
model = tf.keras.Sequential()

#change the input shape if you have sequences less long
model.add(layers.Conv1D(filters=64, kernel_size=8, activation='relu',input_shape=(3198,1)))
model.add(layers.MaxPool1D(strides=5))
model.add(layers.BatchNormalization())


model.add(layers.Conv1D(filters=128, kernel_size=6, activation='relu'))
model.add(layers.MaxPool1D(strides=5))
model.add(layers.BatchNormalization())


model.add(layers.Conv1D(filters=256, kernel_size=4, activation='relu'))
model.add(layers.MaxPool1D(strides=5))
model.add(layers.BatchNormalization())


model.add(layers.Flatten())
model.add(layers.Dropout(0.3))


model.add(layers.Dense(500, activation='relu'))
model.add(layers.Dropout(0.3))

model.add(layers.Dense(250, activation='relu'))


model.add(layers.Dense(50, activation = 'relu'))


model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the Model
# Fit the model
model.fit(data_train_500_norm[:,1:],data_train_500_norm[:,0], epochs=100, batch_size=16, verbose=1)

# Evaluate the Model on Test Data
loss, accuracy = model.evaluate(data_test[:,1:],data_test[:,0], verbose=0)
print(f'Test Accuracy: {accuracy * 100:.2f}%')

Epoch 1/100
 99/318 ━━━━━━━━━━━━━━━━━━━━ 41s 188ms/step - accuracy: 0.0000e+00 - loss: 0.0332

KeyboardInterrupt: 